# Tutorial 3: Learning

In [1]:
import libspn as spn
import tensorflow as tf


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



### Building a Test Graph with Random Weights

In [2]:
indicators_x = spn.IndicatorLeaf(num_vars=2, num_vals=2, name="indicators_x")
sum_11 = spn.Sum((indicators_x, [0,1]), name="sum_11")
sum_12 = spn.Sum((indicators_x, [0,1]), name="sum_12")
sum_21 = spn.Sum((indicators_x, [2,3]), name="sum_21")
sum_22 = spn.Sum((indicators_x, [2,3]), name="sum_22")
prod_1 = spn.Product(sum_11, sum_21, name="prod_1")
prod_2 = spn.Product(sum_11, sum_22, name="prod_2")
prod_3 = spn.Product(sum_12, sum_22, name="prod_3")
root = spn.Sum(prod_1, prod_2, prod_3, name="root")
root = spn.convert_to_layer_nodes(root)
latent_indicator_y = root.generate_latent_indicators(name="latent_indicator_y")
spn.generate_weights(root, initializer=tf.initializers.random_uniform(1.0, 2.0))

Instructions for updating:
Colocations handled automatically by placer.


[WARNING] [tensorflow:new_func] From /home/jos/.local/lib/python3.5/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


### Visualizing the SPN Graph

In [3]:
spn.display_spn_graph(root)

### Specify Training Data

In [4]:
indicators_x_arr=[[0,0],[0,0],[1,1],[1,1],[1,1],[0,1],[0,1],[0,1]]
latent_indicator_y_arr=[[-1]] * len(indicators_x_arr)

### Add Learning Ops

In [52]:
init_weights = spn.initialize_weights(root)
learning = spn.EMLearning(root, initial_accum_value=0.01, sample_prob=0.1, sample=True)
init_learning = learning.reset_accumulators()
accumulate_updates = learning.accumulate_updates()
update_spn = learning.update_spn()
likelihood = tf.reduce_mean(learning.value.values[root])

### Run Learning

In [56]:
epoch = 0
with tf.Session() as sess:
    sess.run(init_learning)
    sess.run(init_weights)
    for _ in range(10):
        likelihood_arr, _, *nodes_out = sess.run(
            [likelihood, accumulate_updates],
            feed_dict={indicators_x: indicators_x_arr, latent_indicator_y: latent_indicator_y_arr})
        print("Avg. Likelihood: %s" % (likelihood_arr))
        sess.run(update_spn)
    

Avg. Likelihood: -1.4000051
Avg. Likelihood: -1.0844302
Avg. Likelihood: -1.0833168
Avg. Likelihood: -1.0829438
Avg. Likelihood: -1.1476172
Avg. Likelihood: -1.1472332
Avg. Likelihood: -1.1359968
Avg. Likelihood: -1.1414859
Avg. Likelihood: -1.1479024
Avg. Likelihood: -1.1348104
